In [1]:
import torch

# Encoding
def encode(list_of_strings, pad_token_id=0):
    max_length = 65536 #max([len(string) for string in list_of_strings])

    # create emtpy tensors
    attention_masks = torch.zeros((len(list_of_strings), max_length), dtype=torch.long)
    input_ids = torch.full((len(list_of_strings), max_length), pad_token_id, dtype=torch.long)

    for idx, string in enumerate(list_of_strings):
        # make sure string is in byte format
        if not isinstance(string, bytes):
            string = str.encode(string)

        input_ids[idx, :len(string)] = torch.tensor([x + 2 for x in string])
        attention_masks[idx, :len(string)] = 1

    return input_ids, attention_masks

# Decoding
def decode(outputs_ids):
    decoded_outputs = []
    for output_ids in outputs_ids.tolist():
        # transform id back to char IDs < 2 are simply transformed to ""
        decoded_outputs.append("".join([chr(x - 2) if x > 1 else "" for x in output_ids]))
    return decoded_outputs

In [2]:
import pandas as pd
from transformers import ReformerModelWithLMHead

train_dataset = pd.read_pickle('train.pkl')
val_dataset = pd.read_pickle('dev.pkl')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = ReformerModelWithLMHead.from_pretrained("google/reformer-enwik8").to(device)
# model = ReformerModelWithLMHead.from_pretrained("patrickvonplaten/reformer-random").to(device)

In [3]:
# optim.zero_grad()
from torch import optim

model.to(device)
model.train()

batch_in = [train_dataset.iloc[0]['input_text']]
batch_gold = [train_dataset.iloc[0]['target_text']]


input_ids, attention_masks = encode(batch_in)
input_ids = input_ids.to(model.device)
attention_masks = attention_masks.to(model.device)

gold_ids, _ = encode(batch_gold)
gold_ids = gold_ids.cuda()

print(input_ids.device)
print(gold_ids.device)
print(model.device)

# optimizer = optim.Adam(model.parameters(), lr=1e-5)

for i in range(2):
    outputs = model(input_ids=input_ids, attention_mask=attention_masks, labels=gold_ids)
    print(outputs.loss)
#     loss = outputs.loss
#     loss.backward()
# loss = outputs[0]
# loss.backward()
# optim.step()

# model.eval()

cuda:0
cuda:0
cuda:0


RuntimeError: CUDA out of memory. Tried to allocate 4.00 GiB (GPU 0; 14.76 GiB total capacity; 11.68 GiB already allocated; 1.18 GiB free; 12.88 GiB reserved in total by PyTorch)

In [4]:
model.device

device(type='cuda', index=0)

In [2]:
from transformers import ReformerModelWithLMHead

model = ReformerModelWithLMHead.from_pretrained("google/reformer-enwik8")
encoded, attention_masks = encode(["In 1965, Brooks left IBM to found the Department of"])
decode(model.generate(encoded, do_sample=True, max_length=150))

# gives:
# In 1965, Brooks left IBM to found the Department of Journalism in 1968. IBM had jurisdiction himself in 1980, while Brooks resolved, nevertheless thro

['In 1965, Brooks left IBM to found the Department of Foreign Affairs in [[1968]] and in [[1971]] the BSA faced [[Rondie]], [[Panama]], [[Vancouver]], [']

In [4]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=10,  # batch size per device during training
    per_device_eval_batch_size=10,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

In [ ]:
trainer.train()

In [9]:
train_dataset.iloc[0]

input_text            she beowright
target_text    She'll be all right.
Name: 0, dtype: object

In [20]:
print(outputs.loss)

tensor(1.9193, grad_fn=<NllLossBackward>)


In [21]:
print(outputs.logits)

tensor([[[-13.0435, -16.7595, -16.9514,  ..., -16.7346, -16.9839, -16.9395],
         [-11.6798, -16.7449, -16.9717,  ..., -16.7032, -17.1898, -17.1265],
         [-14.4155, -19.2803, -19.2556,  ..., -19.1648, -19.6300, -19.5507],
         ...,
         [-21.8854, -23.0179, -23.4852,  ..., -23.1193, -23.0972, -23.1477],
         [-12.8894, -20.8777, -20.6602,  ..., -20.6046, -20.4435, -20.6265],
         [-14.1225, -19.3963, -19.3595,  ..., -19.1962, -19.1647, -19.5313]]],
       grad_fn=<AddBackward0>)


In [22]:
print(outputs.hidden_states)

None


In [25]:
print(outputs.attentions)

None


In [29]:
decode(outputs.logits)

TypeError: '>' not supported between instances of 'list' and 'int'